In [ ]:
## This file implements neural networks and logistic regression on pyopresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination 
## of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 97.71% for combination data and 100% for over-sampling data
## after regularization for combination data.
## For logistic regression, the accuracy is 97.71% for combination data and 100% for over-sampling data.
## For random forest, the accuracy is 100% for both combination and over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99% for combination data, and 99.09% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/pyopresabs_quant.csv')
df.shape

(255, 539)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.158375
1      0.157875
2      0.310875
3      0.160125
4      0.160500
5      0.205000
6      0.158250
7      0.158375
8      0.166000
9      0.157750
10     0.159375
11     0.159000
12     0.161000
13     0.161875
14     0.157875
15     0.157125
16     0.305000
17     0.281250
18     0.244625
19     0.156875
20     0.169625
21     0.282500
22     0.156750
23     0.161875
24     0.525750
25     0.155375
26     0.171375
27     0.156500
28     0.158625
29     0.288500
         ...   
225    0.179000
226    0.333889
227    0.400222
228    0.499444
229    0.523000
230    0.267250
231    0.209000
232    0.243625
233    0.201875
234    0.160000
235    0.184250
236    0.178000
237    0.157000
238    0.158250
239    0.201000
240    0.232750
241    0.174250
242    0.159250
243    0.160125
244    0.154000
245    0.156875
246    0.233375
247    0.157375
248    0.158375
249    0.158625
250    0.231250
251    0.174625
252    0.157000
253    0.167625
254    0.163000
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.158375
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.157875
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.310875
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.160125
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.160500


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

0    237
1     18
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 539)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 538)

In [12]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8666,group_8667,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 537) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 201), (1, 234)]


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 237), (1, 237)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 304 samples, validate on 131 samples
Epoch 1/1000
304/304 [==============================] - 0s 431us/step - loss: 0.6732 - accuracy: 0.5691 - val_loss: 0.6019 - val_accuracy: 0.6641
Epoch 2/1000
304/304 [==============================] - 0s 88us/step - loss: 0.5874 - accuracy: 0.6875 - val_loss: 0.5520 - val_accuracy: 0.7863
Epoch 3/1000
304/304 [==============================] - 0s 116us/step - loss: 0.5431 - accuracy: 0.7500 - val_loss: 0.5141 - val_accuracy: 0.8015
Epoch 4/1000
304/304 [==============================] - 0s 108us/step - loss: 0.5091 - accuracy: 0.7928 - val_loss: 0.4840 - val_accuracy: 0.8168
Epoch 5/1000
304/304 [==============================] - 0s 121us/step - loss: 0.4810 - accuracy: 0.8059 - val_loss: 0.4591 - val_accuracy: 0.8244
Epoch 6/1000
304/304 [==============================] - 0s 113us/step - loss: 0.4566 - accuracy: 0.8158 - val_loss: 0.4375 - val_accuracy: 0.8550
Epoch 7/1000
304/304 [==============================] - 0s 114us/step - loss: 0

304/304 [==============================] - 0s 126us/step - loss: 0.0912 - accuracy: 0.9934 - val_loss: 0.1323 - val_accuracy: 0.9618
Epoch 57/1000
304/304 [==============================] - 0s 108us/step - loss: 0.0891 - accuracy: 0.9967 - val_loss: 0.1311 - val_accuracy: 0.9618
Epoch 58/1000
304/304 [==============================] - 0s 113us/step - loss: 0.0872 - accuracy: 0.9934 - val_loss: 0.1294 - val_accuracy: 0.9695
Epoch 59/1000
304/304 [==============================] - 0s 149us/step - loss: 0.0853 - accuracy: 0.9967 - val_loss: 0.1285 - val_accuracy: 0.9618
Epoch 60/1000
304/304 [==============================] - 0s 118us/step - loss: 0.0838 - accuracy: 0.9934 - val_loss: 0.1267 - val_accuracy: 0.9695
Epoch 61/1000
304/304 [==============================] - 0s 104us/step - loss: 0.0819 - accuracy: 0.9967 - val_loss: 0.1255 - val_accuracy: 0.9695
Epoch 62/1000
304/304 [==============================] - 0s 96us/step - loss: 0.0803 - accuracy: 0.9967 - val_loss: 0.1247 - val_acc

304/304 [==============================] - 0s 138us/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 0.0898 - val_accuracy: 0.9847
Epoch 112/1000
304/304 [==============================] - 0s 103us/step - loss: 0.0366 - accuracy: 1.0000 - val_loss: 0.0895 - val_accuracy: 0.9847
Epoch 113/1000
304/304 [==============================] - 0s 97us/step - loss: 0.0361 - accuracy: 1.0000 - val_loss: 0.0895 - val_accuracy: 0.9847
Epoch 114/1000
304/304 [==============================] - 0s 119us/step - loss: 0.0357 - accuracy: 1.0000 - val_loss: 0.0892 - val_accuracy: 0.9847
Epoch 115/1000
304/304 [==============================] - 0s 96us/step - loss: 0.0353 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9847
Epoch 116/1000
304/304 [==============================] - 0s 115us/step - loss: 0.0349 - accuracy: 1.0000 - val_loss: 0.0887 - val_accuracy: 0.9847
Epoch 117/1000
304/304 [==============================] - 0s 110us/step - loss: 0.0344 - accuracy: 1.0000 - val_loss: 0.0883 - va

304/304 [==============================] - 0s 81us/step - loss: 0.0212 - accuracy: 1.0000 - val_loss: 0.0767 - val_accuracy: 0.9847
Epoch 167/1000
304/304 [==============================] - 0s 94us/step - loss: 0.0210 - accuracy: 1.0000 - val_loss: 0.0767 - val_accuracy: 0.9847
Epoch 168/1000
304/304 [==============================] - 0s 118us/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0765 - val_accuracy: 0.9847
Epoch 169/1000
304/304 [==============================] - 0s 88us/step - loss: 0.0207 - accuracy: 1.0000 - val_loss: 0.0763 - val_accuracy: 0.9847
Epoch 170/1000
304/304 [==============================] - 0s 87us/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 0.0762 - val_accuracy: 0.9847
Epoch 171/1000
304/304 [==============================] - 0s 115us/step - loss: 0.0203 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9847
Epoch 172/1000
304/304 [==============================] - 0s 88us/step - loss: 0.0202 - accuracy: 1.0000 - val_loss: 0.0758 - val_a

Epoch 222/1000
304/304 [==============================] - 0s 84us/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 223/1000
304/304 [==============================] - 0s 113us/step - loss: 0.0141 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 224/1000
304/304 [==============================] - 0s 93us/step - loss: 0.0140 - accuracy: 1.0000 - val_loss: 0.0706 - val_accuracy: 0.9847
Epoch 225/1000
304/304 [==============================] - 0s 98us/step - loss: 0.0139 - accuracy: 1.0000 - val_loss: 0.0705 - val_accuracy: 0.9847
Epoch 226/1000
304/304 [==============================] - 0s 102us/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9847
Epoch 227/1000
304/304 [==============================] - 0s 93us/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9847
Epoch 228/1000
304/304 [==============================] - 0s 94us/step - loss: 0.0137 - accuracy: 1.0000 - val_loss:

...

304/304 [==============================] - 0s 105us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 779/1000
304/304 [==============================] - 0s 121us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 780/1000
304/304 [==============================] - 0s 144us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 781/1000
304/304 [==============================] - 0s 194us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0615 - val_accuracy: 0.9847
Epoch 782/1000
304/304 [==============================] - 0s 177us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 783/1000
304/304 [==============================] - 0s 153us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 784/1000
304/304 [==============================] - 0s 292us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0616 - 

304/304 [==============================] - 0s 146us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 834/1000
304/304 [==============================] - 0s 93us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 835/1000
304/304 [==============================] - 0s 98us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 836/1000
304/304 [==============================] - 0s 119us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 837/1000
304/304 [==============================] - 0s 124us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 838/1000
304/304 [==============================] - 0s 133us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 839/1000
304/304 [==============================] - 0s 95us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0616 - val

304/304 [==============================] - 0s 94us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 889/1000
304/304 [==============================] - 0s 205us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 890/1000
304/304 [==============================] - 0s 146us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 891/1000
304/304 [==============================] - 0s 128us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 892/1000
304/304 [==============================] - 0s 127us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 893/1000
304/304 [==============================] - 0s 115us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 894/1000
304/304 [==============================] - 0s 117us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0616 - v

304/304 [==============================] - 0s 114us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 944/1000
304/304 [==============================] - 0s 112us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 945/1000
304/304 [==============================] - 0s 87us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 946/1000
304/304 [==============================] - 0s 89us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0616 - val_accuracy: 0.9847
Epoch 947/1000
304/304 [==============================] - 0s 93us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 948/1000
304/304 [==============================] - 0s 100us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 949/1000
304/304 [==============================] - 0s 100us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0617 - val

304/304 [==============================] - 0s 85us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 999/1000
304/304 [==============================] - 0s 96us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847
Epoch 1000/1000
304/304 [==============================] - 0s 98us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0617 - val_accuracy: 0.9847


In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

131/131 [==============================] - 0s 80us/step
combination test accuracy: 98.47%


In [24]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [25]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 304 samples, validate on 131 samples
Epoch 1/1000
304/304 [==============================] - 0s 559us/step - loss: 2.6363 - accuracy: 0.6349 - val_loss: 0.7668 - val_accuracy: 0.6489
Epoch 2/1000
304/304 [==============================] - 0s 99us/step - loss: 2.4693 - accuracy: 0.7270 - val_loss: 0.7198 - val_accuracy: 0.7023
Epoch 3/1000
304/304 [==============================] - 0s 129us/step - loss: 2.1687 - accuracy: 0.7599 - val_loss: 0.6835 - val_accuracy: 0.7405
Epoch 4/1000
304/304 [==============================] - 0s 142us/step - loss: 2.4337 - accuracy: 0.7566 - val_loss: 0.6560 - val_accuracy: 0.7634
Epoch 5/1000
304/304 [==============================] - 0s 126us/step - loss: 1.6052 - accuracy: 0.8289 - val_loss: 0.6304 - val_accuracy: 0.8550
Epoch 6/1000
304/304 [==============================] - 0s 110us/step - loss: 1.5949 - accuracy: 0.8388 - val_loss: 0.6111 - val_accuracy: 0.8550
Epoch 7/1000
304/304 [==============================] - 0s 137us/step - loss: 2

304/304 [==============================] - 0s 176us/step - loss: 2.1223 - accuracy: 0.8717 - val_loss: 0.3404 - val_accuracy: 0.9695
Epoch 57/1000
304/304 [==============================] - 0s 148us/step - loss: 1.8709 - accuracy: 0.8914 - val_loss: 0.3383 - val_accuracy: 0.9695
Epoch 58/1000
304/304 [==============================] - 0s 126us/step - loss: 1.6129 - accuracy: 0.9079 - val_loss: 0.3361 - val_accuracy: 0.9695
Epoch 59/1000
304/304 [==============================] - 0s 106us/step - loss: 1.8642 - accuracy: 0.8914 - val_loss: 0.3337 - val_accuracy: 0.9847
Epoch 60/1000
304/304 [==============================] - 0s 103us/step - loss: 1.3057 - accuracy: 0.9276 - val_loss: 0.3315 - val_accuracy: 0.9847
Epoch 61/1000
304/304 [==============================] - 0s 101us/step - loss: 1.7627 - accuracy: 0.8980 - val_loss: 0.3294 - val_accuracy: 0.9847
Epoch 62/1000
304/304 [==============================] - 0s 97us/step - loss: 2.0115 - accuracy: 0.8816 - val_loss: 0.3283 - val_acc

Epoch 112/1000
304/304 [==============================] - 0s 108us/step - loss: 1.8612 - accuracy: 0.8882 - val_loss: 0.2767 - val_accuracy: 0.9924
Epoch 113/1000
304/304 [==============================] - 0s 108us/step - loss: 1.5562 - accuracy: 0.9079 - val_loss: 0.2757 - val_accuracy: 0.9924
Epoch 114/1000
304/304 [==============================] - 0s 103us/step - loss: 1.8573 - accuracy: 0.8882 - val_loss: 0.2758 - val_accuracy: 0.9924
Epoch 115/1000
304/304 [==============================] - 0s 110us/step - loss: 1.7542 - accuracy: 0.8947 - val_loss: 0.2754 - val_accuracy: 0.9924
Epoch 116/1000
304/304 [==============================] - 0s 99us/step - loss: 2.3156 - accuracy: 0.8586 - val_loss: 0.2743 - val_accuracy: 0.9924
Epoch 117/1000
304/304 [==============================] - 0s 101us/step - loss: 1.5520 - accuracy: 0.9079 - val_loss: 0.2742 - val_accuracy: 0.9924
Epoch 118/1000
304/304 [==============================] - 0s 95us/step - loss: 1.9080 - accuracy: 0.8849 - val_lo

304/304 [==============================] - 0s 92us/step - loss: 1.7858 - accuracy: 0.8914 - val_loss: 0.2541 - val_accuracy: 0.9924
Epoch 168/1000
304/304 [==============================] - 0s 102us/step - loss: 2.3447 - accuracy: 0.8553 - val_loss: 0.2540 - val_accuracy: 0.9924
Epoch 169/1000
304/304 [==============================] - 0s 111us/step - loss: 1.9380 - accuracy: 0.8816 - val_loss: 0.2537 - val_accuracy: 0.9924
Epoch 170/1000
304/304 [==============================] - 0s 105us/step - loss: 1.6349 - accuracy: 0.9013 - val_loss: 0.2531 - val_accuracy: 0.9924
Epoch 171/1000
304/304 [==============================] - 0s 114us/step - loss: 1.6340 - accuracy: 0.9013 - val_loss: 0.2527 - val_accuracy: 0.9924
Epoch 172/1000
304/304 [==============================] - 0s 108us/step - loss: 2.1919 - accuracy: 0.8651 - val_loss: 0.2527 - val_accuracy: 0.9924
Epoch 173/1000
304/304 [==============================] - 0s 94us/step - loss: 2.1901 - accuracy: 0.8651 - val_loss: 0.2524 - va

304/304 [==============================] - 0s 105us/step - loss: 1.7208 - accuracy: 0.8947 - val_loss: 0.2412 - val_accuracy: 0.9924
Epoch 223/1000
304/304 [==============================] - 0s 114us/step - loss: 1.5670 - accuracy: 0.9046 - val_loss: 0.2412 - val_accuracy: 0.9924
Epoch 224/1000
304/304 [==============================] - 0s 109us/step - loss: 2.0237 - accuracy: 0.8750 - val_loss: 0.2410 - val_accuracy: 0.9924
Epoch 225/1000
304/304 [==============================] - 0s 105us/step - loss: 1.5174 - accuracy: 0.9079 - val_loss: 0.2405 - val_accuracy: 0.9924
Epoch 226/1000
304/304 [==============================] - 0s 102us/step - loss: 1.7672 - accuracy: 0.8914 - val_loss: 0.2407 - val_accuracy: 0.9924
Epoch 227/1000
304/304 [==============================] - 0s 99us/step - loss: 1.3620 - accuracy: 0.9178 - val_loss: 0.2404 - val_accuracy: 0.9924
Epoch 228/1000
304/304 [==============================] - 0s 108us/step - loss: 2.1269 - accuracy: 0.8684 - val_loss: 0.2403 - v

...

304/304 [==============================] - 0s 105us/step - loss: 1.3741 - accuracy: 0.9145 - val_loss: 0.2055 - val_accuracy: 0.9771
Epoch 773/1000
304/304 [==============================] - 0s 111us/step - loss: 2.0846 - accuracy: 0.8684 - val_loss: 0.2048 - val_accuracy: 0.9771
Epoch 774/1000
304/304 [==============================] - 0s 105us/step - loss: 1.8822 - accuracy: 0.8816 - val_loss: 0.2047 - val_accuracy: 0.9771
Epoch 775/1000
304/304 [==============================] - 0s 98us/step - loss: 1.7804 - accuracy: 0.8882 - val_loss: 0.2046 - val_accuracy: 0.9771
Epoch 776/1000
304/304 [==============================] - 0s 103us/step - loss: 1.4244 - accuracy: 0.9112 - val_loss: 0.2047 - val_accuracy: 0.9771
Epoch 777/1000
304/304 [==============================] - 0s 123us/step - loss: 1.8817 - accuracy: 0.8816 - val_loss: 0.2043 - val_accuracy: 0.9771
Epoch 778/1000
304/304 [==============================] - 0s 125us/step - loss: 2.2369 - accuracy: 0.8586 - val_loss: 0.2041 - v

304/304 [==============================] - 0s 100us/step - loss: 1.8800 - accuracy: 0.8816 - val_loss: 0.2042 - val_accuracy: 0.9771
Epoch 828/1000
304/304 [==============================] - 0s 110us/step - loss: 1.6265 - accuracy: 0.8980 - val_loss: 0.2040 - val_accuracy: 0.9771
Epoch 829/1000
304/304 [==============================] - 0s 121us/step - loss: 1.5240 - accuracy: 0.9046 - val_loss: 0.2035 - val_accuracy: 0.9771
Epoch 830/1000
304/304 [==============================] - 0s 90us/step - loss: 1.7283 - accuracy: 0.8914 - val_loss: 0.2033 - val_accuracy: 0.9771
Epoch 831/1000
304/304 [==============================] - 0s 102us/step - loss: 2.4373 - accuracy: 0.8454 - val_loss: 0.2039 - val_accuracy: 0.9771
Epoch 832/1000
304/304 [==============================] - 0s 102us/step - loss: 1.3214 - accuracy: 0.9178 - val_loss: 0.2033 - val_accuracy: 0.9771
Epoch 833/1000
304/304 [==============================] - 0s 97us/step - loss: 1.5244 - accuracy: 0.9046 - val_loss: 0.2036 - va

304/304 [==============================] - 0s 102us/step - loss: 1.1669 - accuracy: 0.9276 - val_loss: 0.2027 - val_accuracy: 0.9771
Epoch 883/1000
304/304 [==============================] - 0s 119us/step - loss: 1.9292 - accuracy: 0.8783 - val_loss: 0.2024 - val_accuracy: 0.9771
Epoch 884/1000
304/304 [==============================] - 0s 93us/step - loss: 2.0311 - accuracy: 0.8717 - val_loss: 0.2023 - val_accuracy: 0.9771
Epoch 885/1000
304/304 [==============================] - 0s 103us/step - loss: 1.3695 - accuracy: 0.9145 - val_loss: 0.2026 - val_accuracy: 0.9771
Epoch 886/1000
304/304 [==============================] - 0s 99us/step - loss: 1.2701 - accuracy: 0.9211 - val_loss: 0.2024 - val_accuracy: 0.9771
Epoch 887/1000
304/304 [==============================] - 0s 105us/step - loss: 1.8775 - accuracy: 0.8816 - val_loss: 0.2026 - val_accuracy: 0.9771
Epoch 888/1000
304/304 [==============================] - 0s 105us/step - loss: 1.7261 - accuracy: 0.8914 - val_loss: 0.2023 - va

304/304 [==============================] - 0s 105us/step - loss: 1.6234 - accuracy: 0.8980 - val_loss: 0.2005 - val_accuracy: 0.9771
Epoch 938/1000
304/304 [==============================] - 0s 114us/step - loss: 1.7248 - accuracy: 0.8914 - val_loss: 0.1999 - val_accuracy: 0.9771
Epoch 939/1000
304/304 [==============================] - 0s 109us/step - loss: 1.5212 - accuracy: 0.9046 - val_loss: 0.2012 - val_accuracy: 0.9771
Epoch 940/1000
304/304 [==============================] - 0s 124us/step - loss: 1.4203 - accuracy: 0.9112 - val_loss: 0.2007 - val_accuracy: 0.9771
Epoch 941/1000
304/304 [==============================] - 0s 90us/step - loss: 1.6229 - accuracy: 0.8980 - val_loss: 0.2011 - val_accuracy: 0.9771
Epoch 942/1000
304/304 [==============================] - 0s 100us/step - loss: 2.1306 - accuracy: 0.8651 - val_loss: 0.2009 - val_accuracy: 0.9771
Epoch 943/1000
304/304 [==============================] - 0s 102us/step - loss: 1.2677 - accuracy: 0.9211 - val_loss: 0.2004 - v

304/304 [==============================] - 0s 103us/step - loss: 1.7232 - accuracy: 0.8914 - val_loss: 0.1996 - val_accuracy: 0.9771
Epoch 993/1000
304/304 [==============================] - 0s 108us/step - loss: 1.9772 - accuracy: 0.8750 - val_loss: 0.1991 - val_accuracy: 0.9771
Epoch 994/1000
304/304 [==============================] - 0s 108us/step - loss: 2.0791 - accuracy: 0.8684 - val_loss: 0.1997 - val_accuracy: 0.9771
Epoch 995/1000
304/304 [==============================] - 0s 100us/step - loss: 1.5711 - accuracy: 0.9013 - val_loss: 0.1991 - val_accuracy: 0.9771
Epoch 996/1000
304/304 [==============================] - 0s 105us/step - loss: 1.6217 - accuracy: 0.8980 - val_loss: 0.1996 - val_accuracy: 0.9771
Epoch 997/1000
304/304 [==============================] - 0s 168us/step - loss: 1.7734 - accuracy: 0.8882 - val_loss: 0.1996 - val_accuracy: 0.9771
Epoch 998/1000
304/304 [==============================] - 0s 106us/step - loss: 1.5196 - accuracy: 0.9046 - val_loss: 0.1994 - 

In [27]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

131/131 [==============================] - 0s 81us/step
combination test accuracy: 97.71%


In [33]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [34]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 336us/step - loss: 0.7318 - accuracy: 0.4834 - val_loss: 0.6425 - val_accuracy: 0.6643
Epoch 2/1000
331/331 [==============================] - 0s 83us/step - loss: 0.6471 - accuracy: 0.6435 - val_loss: 0.5902 - val_accuracy: 0.7273
Epoch 3/1000
331/331 [==============================] - 0s 109us/step - loss: 0.6013 - accuracy: 0.6888 - val_loss: 0.5555 - val_accuracy: 0.7413
Epoch 4/1000
331/331 [==============================] - 0s 119us/step - loss: 0.5642 - accuracy: 0.7251 - val_loss: 0.5267 - val_accuracy: 0.7762
Epoch 5/1000
331/331 [==============================] - 0s 96us/step - loss: 0.5294 - accuracy: 0.7613 - val_loss: 0.4993 - val_accuracy: 0.8462
Epoch 6/1000
331/331 [==============================] - 0s 105us/step - loss: 0.5026 - accuracy: 0.8187 - val_loss: 0.4781 - val_accuracy: 0.8601
Epoch 7/1000
331/331 [==============================] - 0s 111us/step - loss: 0.

331/331 [==============================] - 0s 114us/step - loss: 0.1335 - accuracy: 0.9879 - val_loss: 0.1656 - val_accuracy: 0.9790
Epoch 57/1000
331/331 [==============================] - 0s 99us/step - loss: 0.1311 - accuracy: 0.9879 - val_loss: 0.1635 - val_accuracy: 0.9790
Epoch 58/1000
331/331 [==============================] - 0s 112us/step - loss: 0.1290 - accuracy: 0.9849 - val_loss: 0.1611 - val_accuracy: 0.9790
Epoch 59/1000
331/331 [==============================] - 0s 97us/step - loss: 0.1266 - accuracy: 0.9879 - val_loss: 0.1598 - val_accuracy: 0.9790
Epoch 60/1000
331/331 [==============================] - 0s 96us/step - loss: 0.1246 - accuracy: 0.9909 - val_loss: 0.1557 - val_accuracy: 0.9790
Epoch 61/1000
331/331 [==============================] - 0s 97us/step - loss: 0.1220 - accuracy: 0.9879 - val_loss: 0.1546 - val_accuracy: 0.9790
Epoch 62/1000
331/331 [==============================] - 0s 111us/step - loss: 0.1205 - accuracy: 0.9909 - val_loss: 0.1529 - val_accura

331/331 [==============================] - 0s 105us/step - loss: 0.0606 - accuracy: 0.9940 - val_loss: 0.0840 - val_accuracy: 0.9860
Epoch 113/1000
331/331 [==============================] - 0s 104us/step - loss: 0.0602 - accuracy: 0.9940 - val_loss: 0.0819 - val_accuracy: 0.9860
Epoch 114/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0595 - accuracy: 0.9940 - val_loss: 0.0813 - val_accuracy: 0.9860
Epoch 115/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0587 - accuracy: 0.9940 - val_loss: 0.0803 - val_accuracy: 0.9860
Epoch 116/1000
331/331 [==============================] - 0s 94us/step - loss: 0.0582 - accuracy: 0.9940 - val_loss: 0.0790 - val_accuracy: 0.9860
Epoch 117/1000
331/331 [==============================] - 0s 96us/step - loss: 0.0577 - accuracy: 0.9940 - val_loss: 0.0778 - val_accuracy: 0.9860
Epoch 118/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0571 - accuracy: 0.9940 - val_loss: 0.0769 - val_a

Epoch 168/1000
331/331 [==============================] - 0s 102us/step - loss: 0.0363 - accuracy: 0.9940 - val_loss: 0.0521 - val_accuracy: 1.0000
Epoch 169/1000
331/331 [==============================] - 0s 90us/step - loss: 0.0361 - accuracy: 0.9940 - val_loss: 0.0515 - val_accuracy: 1.0000
Epoch 170/1000
331/331 [==============================] - 0s 93us/step - loss: 0.0358 - accuracy: 0.9940 - val_loss: 0.0510 - val_accuracy: 1.0000
Epoch 171/1000
331/331 [==============================] - 0s 86us/step - loss: 0.0355 - accuracy: 0.9940 - val_loss: 0.0505 - val_accuracy: 1.0000
Epoch 172/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0352 - accuracy: 0.9940 - val_loss: 0.0509 - val_accuracy: 1.0000
Epoch 173/1000
331/331 [==============================] - 0s 87us/step - loss: 0.0349 - accuracy: 0.9940 - val_loss: 0.0507 - val_accuracy: 1.0000
Epoch 174/1000
331/331 [==============================] - 0s 84us/step - loss: 0.0346 - accuracy: 0.9940 - val_loss: 

Epoch 224/1000
331/331 [==============================] - 0s 90us/step - loss: 0.0243 - accuracy: 1.0000 - val_loss: 0.0366 - val_accuracy: 1.0000
Epoch 225/1000
331/331 [==============================] - 0s 92us/step - loss: 0.0242 - accuracy: 1.0000 - val_loss: 0.0365 - val_accuracy: 1.0000
Epoch 226/1000
331/331 [==============================] - 0s 86us/step - loss: 0.0240 - accuracy: 1.0000 - val_loss: 0.0364 - val_accuracy: 1.0000
Epoch 227/1000
331/331 [==============================] - 0s 84us/step - loss: 0.0239 - accuracy: 0.9970 - val_loss: 0.0365 - val_accuracy: 1.0000
Epoch 228/1000
331/331 [==============================] - 0s 103us/step - loss: 0.0238 - accuracy: 1.0000 - val_loss: 0.0361 - val_accuracy: 1.0000
Epoch 229/1000
331/331 [==============================] - 0s 78us/step - loss: 0.0236 - accuracy: 1.0000 - val_loss: 0.0359 - val_accuracy: 1.0000
Epoch 230/1000
331/331 [==============================] - 0s 88us/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 

...

331/331 [==============================] - 0s 106us/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 727/1000
331/331 [==============================] - 0s 129us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 728/1000
331/331 [==============================] - 0s 139us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 729/1000
331/331 [==============================] - 0s 174us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 730/1000
331/331 [==============================] - 0s 85us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 731/1000
331/331 [==============================] - 0s 145us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - val_accuracy: 1.0000
Epoch 732/1000
331/331 [==============================] - 0s 144us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0097 - v

331/331 [==============================] - 0s 94us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 782/1000
331/331 [==============================] - 0s 101us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0091 - val_accuracy: 1.0000
Epoch 783/1000
331/331 [==============================] - 0s 93us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 784/1000
331/331 [==============================] - 0s 98us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 785/1000
331/331 [==============================] - 0s 99us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 786/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 787/1000
331/331 [==============================] - 0s 95us/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0090 - val_ac

Epoch 837/1000
331/331 [==============================] - 0s 107us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 838/1000
331/331 [==============================] - 0s 92us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 839/1000
331/331 [==============================] - 0s 96us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 840/1000
331/331 [==============================] - 0s 93us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 841/1000
331/331 [==============================] - 0s 84us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 842/1000
331/331 [==============================] - 0s 143us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 843/1000
331/331 [==============================] - 0s 153us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss

331/331 [==============================] - 0s 90us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 893/1000
331/331 [==============================] - 0s 81us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 894/1000
331/331 [==============================] - 0s 74us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 895/1000
331/331 [==============================] - 0s 78us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 896/1000
331/331 [==============================] - 0s 80us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 897/1000
331/331 [==============================] - 0s 82us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 898/1000
331/331 [==============================] - 0s 76us/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0080 - val_acc

Epoch 948/1000
331/331 [==============================] - 0s 90us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 949/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 950/1000
331/331 [==============================] - 0s 87us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 951/1000
331/331 [==============================] - 0s 90us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 952/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 953/1000
331/331 [==============================] - 0s 97us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 954/1000
331/331 [==============================] - 0s 86us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0

In [36]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

143/143 [==============================] - 0s 80us/step
over-sampling test accuracy: 100.00%


In [37]:
#### #### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [38]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 331 samples, validate on 143 samples
Epoch 1/1000
331/331 [==============================] - 0s 518us/step - loss: 2.4367 - accuracy: 0.6073 - val_loss: 0.8595 - val_accuracy: 0.6014
Epoch 2/1000
331/331 [==============================] - 0s 103us/step - loss: 2.3790 - accuracy: 0.6314 - val_loss: 0.7889 - val_accuracy: 0.6643
Epoch 3/1000
331/331 [==============================] - 0s 107us/step - loss: 2.2261 - accuracy: 0.6767 - val_loss: 0.7370 - val_accuracy: 0.6853
Epoch 4/1000
331/331 [==============================] - 0s 116us/step - loss: 2.3426 - accuracy: 0.7402 - val_loss: 0.7044 - val_accuracy: 0.6923
Epoch 5/1000
331/331 [==============================] - 0s 103us/step - loss: 2.0822 - accuracy: 0.7674 - val_loss: 0.6788 - val_accuracy: 0.7133
Epoch 6/1000
331/331 [==============================] - 0s 126us/step - loss: 2.6132 - accuracy: 0.7341 - val_loss: 0.6597 - val_accuracy: 0.7063
Epoch 7/1000
331/331 [==============================] - 0s 117us/step - loss: 

331/331 [==============================] - 0s 135us/step - loss: 1.6274 - accuracy: 0.9033 - val_loss: 0.3772 - val_accuracy: 0.9231
Epoch 57/1000
331/331 [==============================] - 0s 116us/step - loss: 2.0806 - accuracy: 0.8731 - val_loss: 0.3767 - val_accuracy: 0.9231
Epoch 58/1000
331/331 [==============================] - 0s 103us/step - loss: 2.1718 - accuracy: 0.8640 - val_loss: 0.3726 - val_accuracy: 0.9231
Epoch 59/1000
331/331 [==============================] - 0s 96us/step - loss: 2.1172 - accuracy: 0.8671 - val_loss: 0.3729 - val_accuracy: 0.9231
Epoch 60/1000
331/331 [==============================] - 0s 119us/step - loss: 1.9530 - accuracy: 0.8761 - val_loss: 0.3672 - val_accuracy: 0.9231
Epoch 61/1000
331/331 [==============================] - 0s 101us/step - loss: 1.6646 - accuracy: 0.8912 - val_loss: 0.3646 - val_accuracy: 0.9441
Epoch 62/1000
331/331 [==============================] - 0s 95us/step - loss: 1.7016 - accuracy: 0.8912 - val_loss: 0.3619 - val_accu

331/331 [==============================] - 0s 97us/step - loss: 1.9253 - accuracy: 0.8792 - val_loss: 0.2905 - val_accuracy: 0.9930
Epoch 113/1000
331/331 [==============================] - 0s 101us/step - loss: 1.6044 - accuracy: 0.8973 - val_loss: 0.2881 - val_accuracy: 0.9930
Epoch 114/1000
331/331 [==============================] - 0s 99us/step - loss: 2.0169 - accuracy: 0.8761 - val_loss: 0.2866 - val_accuracy: 0.9930
Epoch 115/1000
331/331 [==============================] - 0s 96us/step - loss: 1.6940 - accuracy: 0.8912 - val_loss: 0.2862 - val_accuracy: 0.9930
Epoch 116/1000
331/331 [==============================] - 0s 94us/step - loss: 2.0144 - accuracy: 0.8731 - val_loss: 0.2854 - val_accuracy: 0.9930
Epoch 117/1000
331/331 [==============================] - 0s 90us/step - loss: 1.5919 - accuracy: 0.9033 - val_loss: 0.2833 - val_accuracy: 0.9930
Epoch 118/1000
331/331 [==============================] - 0s 121us/step - loss: 1.5496 - accuracy: 0.9033 - val_loss: 0.2843 - val_a

Epoch 168/1000
331/331 [==============================] - 0s 98us/step - loss: 1.7074 - accuracy: 0.8943 - val_loss: 0.2473 - val_accuracy: 1.0000
Epoch 169/1000
331/331 [==============================] - 0s 109us/step - loss: 1.8045 - accuracy: 0.8852 - val_loss: 0.2461 - val_accuracy: 1.0000
Epoch 170/1000
331/331 [==============================] - 0s 104us/step - loss: 1.7555 - accuracy: 0.8882 - val_loss: 0.2466 - val_accuracy: 1.0000
Epoch 171/1000
331/331 [==============================] - 0s 93us/step - loss: 1.2460 - accuracy: 0.9215 - val_loss: 0.2445 - val_accuracy: 1.0000
Epoch 172/1000
331/331 [==============================] - 0s 94us/step - loss: 1.6614 - accuracy: 0.8943 - val_loss: 0.2444 - val_accuracy: 1.0000
Epoch 173/1000
331/331 [==============================] - 0s 90us/step - loss: 2.0366 - accuracy: 0.8701 - val_loss: 0.2445 - val_accuracy: 1.0000
Epoch 174/1000
331/331 [==============================] - 0s 97us/step - loss: 1.9409 - accuracy: 0.8792 - val_loss:

331/331 [==============================] - 0s 92us/step - loss: 1.4151 - accuracy: 0.9124 - val_loss: 0.2239 - val_accuracy: 1.0000
Epoch 224/1000
331/331 [==============================] - 0s 104us/step - loss: 1.7378 - accuracy: 0.8912 - val_loss: 0.2255 - val_accuracy: 1.0000
Epoch 225/1000
331/331 [==============================] - 0s 97us/step - loss: 1.9210 - accuracy: 0.8822 - val_loss: 0.2240 - val_accuracy: 1.0000
Epoch 226/1000
331/331 [==============================] - 0s 93us/step - loss: 1.7385 - accuracy: 0.8882 - val_loss: 0.2235 - val_accuracy: 1.0000
Epoch 227/1000
331/331 [==============================] - 0s 97us/step - loss: 1.9711 - accuracy: 0.8761 - val_loss: 0.2226 - val_accuracy: 1.0000
Epoch 228/1000
331/331 [==============================] - 0s 94us/step - loss: 2.0144 - accuracy: 0.8731 - val_loss: 0.2231 - val_accuracy: 1.0000
Epoch 229/1000
331/331 [==============================] - 0s 92us/step - loss: 1.8336 - accuracy: 0.8852 - val_loss: 0.2281 - val_ac

...

331/331 [==============================] - 0s 98us/step - loss: 1.5501 - accuracy: 0.9033 - val_loss: 0.1712 - val_accuracy: 1.0000
Epoch 779/1000
331/331 [==============================] - 0s 103us/step - loss: 1.6897 - accuracy: 0.8943 - val_loss: 0.1706 - val_accuracy: 1.0000
Epoch 780/1000
331/331 [==============================] - 0s 99us/step - loss: 2.0160 - accuracy: 0.8731 - val_loss: 0.1706 - val_accuracy: 1.0000
Epoch 781/1000
331/331 [==============================] - 0s 91us/step - loss: 1.7349 - accuracy: 0.8912 - val_loss: 0.1711 - val_accuracy: 1.0000
Epoch 782/1000
331/331 [==============================] - 0s 105us/step - loss: 1.7362 - accuracy: 0.8912 - val_loss: 0.1715 - val_accuracy: 1.0000
Epoch 783/1000
331/331 [==============================] - 0s 92us/step - loss: 1.6898 - accuracy: 0.8943 - val_loss: 0.1708 - val_accuracy: 1.0000
Epoch 784/1000
331/331 [==============================] - 0s 90us/step - loss: 1.7364 - accuracy: 0.8912 - val_loss: 0.1710 - val_a

331/331 [==============================] - 0s 94us/step - loss: 1.6878 - accuracy: 0.8943 - val_loss: 0.1684 - val_accuracy: 1.0000
Epoch 834/1000
331/331 [==============================] - 0s 106us/step - loss: 1.7347 - accuracy: 0.8912 - val_loss: 0.1695 - val_accuracy: 1.0000
Epoch 835/1000
331/331 [==============================] - 0s 101us/step - loss: 1.5479 - accuracy: 0.9033 - val_loss: 0.1692 - val_accuracy: 1.0000
Epoch 836/1000
331/331 [==============================] - 0s 98us/step - loss: 1.5947 - accuracy: 0.9003 - val_loss: 0.1692 - val_accuracy: 1.0000
Epoch 837/1000
331/331 [==============================] - 0s 97us/step - loss: 1.3615 - accuracy: 0.9154 - val_loss: 0.1678 - val_accuracy: 1.0000
Epoch 838/1000
331/331 [==============================] - 0s 91us/step - loss: 1.3606 - accuracy: 0.9154 - val_loss: 0.1687 - val_accuracy: 1.0000
Epoch 839/1000
331/331 [==============================] - 0s 98us/step - loss: 1.3619 - accuracy: 0.9154 - val_loss: 0.1689 - val_a

Epoch 889/1000
331/331 [==============================] - 0s 99us/step - loss: 1.4992 - accuracy: 0.9063 - val_loss: 0.1672 - val_accuracy: 1.0000
Epoch 890/1000
331/331 [==============================] - 0s 101us/step - loss: 1.9189 - accuracy: 0.8792 - val_loss: 0.1673 - val_accuracy: 1.0000
Epoch 891/1000
331/331 [==============================] - 0s 99us/step - loss: 1.4997 - accuracy: 0.9063 - val_loss: 0.1671 - val_accuracy: 1.0000
Epoch 892/1000
331/331 [==============================] - 0s 98us/step - loss: 1.4521 - accuracy: 0.9094 - val_loss: 0.1672 - val_accuracy: 1.0000
Epoch 893/1000
331/331 [==============================] - 0s 96us/step - loss: 1.4525 - accuracy: 0.9094 - val_loss: 0.1665 - val_accuracy: 1.0000
Epoch 894/1000
331/331 [==============================] - 0s 99us/step - loss: 1.6395 - accuracy: 0.8973 - val_loss: 0.1671 - val_accuracy: 1.0000
Epoch 895/1000
331/331 [==============================] - 0s 96us/step - loss: 1.6389 - accuracy: 0.8973 - val_loss: 

Epoch 945/1000
331/331 [==============================] - 0s 112us/step - loss: 1.5437 - accuracy: 0.9033 - val_loss: 0.1653 - val_accuracy: 1.0000
Epoch 946/1000
331/331 [==============================] - 0s 91us/step - loss: 1.4033 - accuracy: 0.9124 - val_loss: 0.1650 - val_accuracy: 1.0000
Epoch 947/1000
331/331 [==============================] - 0s 95us/step - loss: 1.8238 - accuracy: 0.8852 - val_loss: 0.1652 - val_accuracy: 1.0000
Epoch 948/1000
331/331 [==============================] - 0s 91us/step - loss: 1.7768 - accuracy: 0.8882 - val_loss: 0.1655 - val_accuracy: 1.0000
Epoch 949/1000
331/331 [==============================] - 0s 94us/step - loss: 1.7312 - accuracy: 0.8912 - val_loss: 0.1649 - val_accuracy: 1.0000
Epoch 950/1000
331/331 [==============================] - 0s 96us/step - loss: 1.6834 - accuracy: 0.8943 - val_loss: 0.1648 - val_accuracy: 1.0000
Epoch 951/1000
331/331 [==============================] - 0s 93us/step - loss: 1.7771 - accuracy: 0.8882 - val_loss: 

331/331 [==============================] - 0s 98us/step - loss: 1.4491 - accuracy: 0.9094 - val_loss: 0.1635 - val_accuracy: 1.0000


In [40]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

143/143 [==============================] - 0s 216us/step
over-sampling test accuracy: 100.00%


In [41]:
############ Logistic Regression ############

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [43]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.71%


In [45]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [47]:
############## Random Forest ##############

In [48]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [50]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [52]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [53]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.   1.   1.   1.   0.95]
0.99


In [54]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[1.         1.         0.98484848 0.98484848 0.98484848]
0.990909090909091
